In [25]:
# Imports
%load_ext autoreload
%autoreload 2
%matplotlib inline
from matplotlib import pyplot as plt
import csv
import importlib
from tqdm import tqdm
from scripts import proj1_helpers, helpers
from scripts import implementation, feature_processing, k_fold, model_linear, model_logistic
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Using TensorFlow backend.


In [2]:
train_path = '../data/train.csv'
test_path  = '../data/test.csv'
output_path = '../data/lr_cv_augm.csv'

In [80]:
# loading data
y, X, idx = proj1_helpers.load_csv_data(train_path)
y_t, X_t, ids_t = proj1_helpers.load_csv_data(test_path)

In [81]:
deg = 3
lambda_ = 2e-4
gamma = 0.1
max_iters = 80

In [119]:
X_p, (x_mean, x_std) = feature_processing.process_X(X, deg)
X_t_p, _ = feature_processing.process_X(X_t, deg, (x_mean, x_std))

 67%|██████▋   | 2/3 [00:02<00:01,  1.27s/it]


In [7]:
n1 = X_p.shape[0]
n2 = X_t_p.shape[0]

In [8]:
#X_p_all = np.vstack((X_p, X_t_p))
#X_p_all_std, _, _ = feature_processing.standardize(X_p_all)

In [ ]:
#X_p = X_p_all_std[:n1,:]
#X_t_p = X_p_all_std[n1:,:]

In [105]:
#X_p, _, _ = feature_processing.standardize(X_p)

In [72]:
assert(X_p.shape[0] == n1)
assert(X_t_p.shape[0] == n2)

In [45]:
#Logistic regression

# Converting -1,1 to 0,1
y_01 = helpers.y_to_01(np.array([y]).T)

In [14]:
np.random.seed(42)
idx_val = np.random.choice(range(X_p.shape[0]), int(X_p.shape[0] * 0.1))
idx_train = np.setdiff1d(range(X_p.shape[0]), idx_val)

In [42]:
# starting point: zero vector
w0 = np.zeros((X.shape[1], 1))
lambda_ = 1e-10
gamma = 0.5

In [ ]:
for i in range(100):
    w0, l = implementation.reg_logistic_regression_newton(y_01[idx_train, :], X_p[idx_train, :], lambda_ = lambda_,
                                                          initial_w = w0, max_iters = 1, gamma = gamma,
                                                          debug = False)
    y_pred = proj1_helpers.predict_labels(w0, X_p[idx_val, :])
    val_acc = np.mean(np.array(y[idx_val].reshape(-1) == np.array(y_pred).reshape(-1)))
    print(i, val_acc)

In [141]:
reg_l2 = 1e-10
hidden_layers = (200, 100, 10)
input_dim = X_p.shape[1]

model = Sequential()
np.random.seed(42)
kernel_regularizer = l2(reg_l2)
# first hidden layer
model.add(Dense(hidden_layers[0], input_dim=input_dim, activation='relu', kernel_regularizer = kernel_regularizer))
# hidden layers
for neurons_n in hidden_layers[1:]:
    model.add(Dense(neurons_n, activation='relu', kernel_regularizer=kernel_regularizer))
#output layer
model.add(Dense(1, activation='sigmoid', kernel_regularizer = kernel_regularizer))
# showing accuracy
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

In [142]:
history = model.fit(x=X_p, y=y_01, batch_size=100000, epochs=1000, validation_split=0.1,
               shuffle=True)

Train on 225000 samples, validate on 25000 samples
Epoch 1/1000
225000/225000 [==============================] - 2s 10us/step - loss: 0.6549 - acc: 0.6347 - val_loss: 0.5979 - val_acc: 0.6806
Epoch 2/1000
225000/225000 [==============================] - 1s 3us/step - loss: 0.5870 - acc: 0.6875 - val_loss: 0.5546 - val_acc: 0.7072
Epoch 3/1000
225000/225000 [==============================] - 1s 3us/step - loss: 0.5459 - acc: 0.7128 - val_loss: 0.5241 - val_acc: 0.7306
Epoch 4/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.5180 - acc: 0.7352 - val_loss: 0.5072 - val_acc: 0.7456
Epoch 5/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.5019 - acc: 0.7517 - val_loss: 0.4957 - val_acc: 0.7552
Epoch 6/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.4904 - acc: 0.7612 - val_loss: 0.4869 - val_acc: 0.7634
Epoch 7/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.4813 - acc: 0.7687 - val

Epoch 59/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3603 - acc: 0.8394 - val_loss: 0.3700 - val_acc: 0.8352
Epoch 60/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3600 - acc: 0.8396 - val_loss: 0.3703 - val_acc: 0.8348
Epoch 61/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3598 - acc: 0.8395 - val_loss: 0.3696 - val_acc: 0.8346
Epoch 62/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3594 - acc: 0.8400 - val_loss: 0.3698 - val_acc: 0.8342
Epoch 63/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3591 - acc: 0.8399 - val_loss: 0.3693 - val_acc: 0.8346
Epoch 64/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3585 - acc: 0.8400 - val_loss: 0.3687 - val_acc: 0.8354
Epoch 65/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3582 - acc: 0.8403 - val_loss: 0.3685 - val_acc: 0.8344
Epoch 66/1000

Epoch 117/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3471 - acc: 0.8458 - val_loss: 0.3631 - val_acc: 0.8370
Epoch 118/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3472 - acc: 0.8453 - val_loss: 0.3627 - val_acc: 0.8367
Epoch 119/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3469 - acc: 0.8456 - val_loss: 0.3634 - val_acc: 0.8358
Epoch 120/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3470 - acc: 0.8459 - val_loss: 0.3636 - val_acc: 0.8371
Epoch 121/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3469 - acc: 0.8457 - val_loss: 0.3634 - val_acc: 0.8362
Epoch 122/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3467 - acc: 0.8460 - val_loss: 0.3627 - val_acc: 0.8368
Epoch 123/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3463 - acc: 0.8462 - val_loss: 0.3625 - val_acc: 0.8373
Epoch 

225000/225000 [==============================] - 1s 4us/step - loss: 0.3405 - acc: 0.8492 - val_loss: 0.3621 - val_acc: 0.8364
Epoch 175/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3406 - acc: 0.8493 - val_loss: 0.3616 - val_acc: 0.8371
Epoch 176/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3404 - acc: 0.8495 - val_loss: 0.3618 - val_acc: 0.8371
Epoch 177/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3402 - acc: 0.8496 - val_loss: 0.3619 - val_acc: 0.8369
Epoch 178/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3399 - acc: 0.8497 - val_loss: 0.3615 - val_acc: 0.8373
Epoch 179/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3398 - acc: 0.8495 - val_loss: 0.3616 - val_acc: 0.8376
Epoch 180/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3396 - acc: 0.8499 - val_loss: 0.3618 - val_acc: 0.8370
Epoch 181/1000
225000

Epoch 232/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3358 - acc: 0.8516 - val_loss: 0.3625 - val_acc: 0.8362
Epoch 233/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3353 - acc: 0.8519 - val_loss: 0.3624 - val_acc: 0.8370
Epoch 234/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3350 - acc: 0.8518 - val_loss: 0.3624 - val_acc: 0.8368
Epoch 235/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3350 - acc: 0.8521 - val_loss: 0.3627 - val_acc: 0.8370
Epoch 236/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3351 - acc: 0.8519 - val_loss: 0.3628 - val_acc: 0.8372
Epoch 237/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3350 - acc: 0.8523 - val_loss: 0.3624 - val_acc: 0.8370
Epoch 238/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3350 - acc: 0.8520 - val_loss: 0.3632 - val_acc: 0.8356
Epoch 

...

225000/225000 [==============================] - 1s 4us/step - loss: 0.3102 - acc: 0.8640 - val_loss: 0.3809 - val_acc: 0.8308
Epoch 750/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3098 - acc: 0.8641 - val_loss: 0.3815 - val_acc: 0.8315
Epoch 751/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3106 - acc: 0.8640 - val_loss: 0.3807 - val_acc: 0.8320
Epoch 752/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3097 - acc: 0.8644 - val_loss: 0.3811 - val_acc: 0.8312
Epoch 753/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3097 - acc: 0.8639 - val_loss: 0.3819 - val_acc: 0.8302
Epoch 754/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3097 - acc: 0.8642 - val_loss: 0.3810 - val_acc: 0.8313
Epoch 755/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3097 - acc: 0.8643 - val_loss: 0.3814 - val_acc: 0.8320
Epoch 756/1000
225000

Epoch 807/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3080 - acc: 0.8649 - val_loss: 0.3829 - val_acc: 0.8316
Epoch 808/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3077 - acc: 0.8651 - val_loss: 0.3830 - val_acc: 0.8317
Epoch 809/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3077 - acc: 0.8651 - val_loss: 0.3834 - val_acc: 0.8303
Epoch 810/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3081 - acc: 0.8649 - val_loss: 0.3832 - val_acc: 0.8318
Epoch 811/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3078 - acc: 0.8653 - val_loss: 0.3834 - val_acc: 0.8313
Epoch 812/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3077 - acc: 0.8650 - val_loss: 0.3838 - val_acc: 0.8295
Epoch 813/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3080 - acc: 0.8649 - val_loss: 0.3841 - val_acc: 0.8314
Epoch 

225000/225000 [==============================] - 1s 4us/step - loss: 0.3076 - acc: 0.8654 - val_loss: 0.3872 - val_acc: 0.8288
Epoch 865/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3074 - acc: 0.8652 - val_loss: 0.3891 - val_acc: 0.8288
Epoch 866/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3086 - acc: 0.8649 - val_loss: 0.3874 - val_acc: 0.8300
Epoch 867/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3077 - acc: 0.8652 - val_loss: 0.3855 - val_acc: 0.8313
Epoch 868/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3066 - acc: 0.8658 - val_loss: 0.3860 - val_acc: 0.8293
Epoch 869/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3064 - acc: 0.8656 - val_loss: 0.3855 - val_acc: 0.8298
Epoch 870/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3063 - acc: 0.8661 - val_loss: 0.3855 - val_acc: 0.8317
Epoch 871/1000
225000

Epoch 922/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3048 - acc: 0.8669 - val_loss: 0.3876 - val_acc: 0.8296
Epoch 923/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3052 - acc: 0.8663 - val_loss: 0.3872 - val_acc: 0.8307
Epoch 924/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3044 - acc: 0.8670 - val_loss: 0.3869 - val_acc: 0.8296
Epoch 925/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3047 - acc: 0.8667 - val_loss: 0.3874 - val_acc: 0.8294
Epoch 926/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3047 - acc: 0.8669 - val_loss: 0.3882 - val_acc: 0.8291
Epoch 927/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3053 - acc: 0.8663 - val_loss: 0.3873 - val_acc: 0.8304
Epoch 928/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3048 - acc: 0.8666 - val_loss: 0.3873 - val_acc: 0.8302
Epoch 

225000/225000 [==============================] - 1s 4us/step - loss: 0.3034 - acc: 0.8675 - val_loss: 0.3890 - val_acc: 0.8298
Epoch 980/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3035 - acc: 0.8676 - val_loss: 0.3899 - val_acc: 0.8301
Epoch 981/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3036 - acc: 0.8671 - val_loss: 0.3897 - val_acc: 0.8301
Epoch 982/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3039 - acc: 0.8673 - val_loss: 0.3888 - val_acc: 0.8299
Epoch 983/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3033 - acc: 0.8675 - val_loss: 0.3891 - val_acc: 0.8300
Epoch 984/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3033 - acc: 0.8671 - val_loss: 0.3910 - val_acc: 0.8282
Epoch 985/1000
225000/225000 [==============================] - 1s 4us/step - loss: 0.3042 - acc: 0.8669 - val_loss: 0.3896 - val_acc: 0.8297
Epoch 986/1000
225000

In [124]:
y_pred = model.predict(X_t_p)

In [132]:
y_pred = 2 * (1. * (y_pred > 0.5) - 0.5)

In [133]:
#y_pred = proj1_helpers.predict_labels(w0 , X_t_p)

In [134]:
proj1_helpers.create_csv_submission(ids_t, y_pred, output_path)

In [14]:
tqdm.get_lock().locks = []
from tqdm import tqdm